Train SVM for each finger pair!\
Data source: \
sliding windowed powers for mu and beta band\
Subject 1

In [1]:
import os
import numpy as np
import h5py
import mne
from scipy import stats
import scipy.io

mne.set_log_level('error')

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [5]:
subject_id = 0
tag = '0_25powers' # contains bad

In [7]:
# Load the dictionary from the HDF5 file
target_dir = 'features'
file_path = os.path.join(target_dir, tag + '_' + cfg['subjects'][subject_id] + '.h5')


data = {}
with h5py.File(file_path, 'r') as h5file:
    for key in h5file.keys():
        data[key] = np.array(h5file[key])

# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 158, 30, 2)
little (50, 158, 30, 2)
middle (50, 158, 30, 2)
ring (50, 158, 30, 2)
thumb (50, 158, 30, 2)


In [8]:
for key, value in data.items():
    data[key] = value.reshape(value.shape[0], -1)

    
# Print the loaded data dictionary
for key, value in data.items():
    print(key, value.shape)

index (50, 9480)
little (50, 9480)
middle (50, 9480)
ring (50, 9480)
thumb (50, 9480)


In [9]:
def train_SVM(finger1, finger2):
    print(f'Training SVM for {finger1} vs {finger2}')

    X = np.concatenate((data[finger1], data[finger2]), axis=0)
    y = np.concatenate((np.ones(data[finger1].shape[0]), np.zeros(data[finger2].shape[0])), axis=0)


    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    tuned_parameters = [{'kernel': ['rbf', 'linear'], 'gamma': [1e-3, 1e-4, 1e-5, 1e-6],
                        'C': [ 1, 10, 100, 1000 ]}]
    grid = GridSearchCV(SVC(), tuned_parameters, cv=StratifiedKFold(n_splits=10), scoring='accuracy')
    grid.fit(X, y)
    acc = grid.best_score_* 100
    print(str(round(acc, 2)))
    print(grid.best_params_)
    return acc

    

In [10]:
accuracies = []
for finger1 in data:
    for finger2 in data:
        if finger1 != finger2:
            accuracies.append(train_SVM(finger1, finger2))
        else:
            break
accucarcies = np.array(accuracies)
print(f'Average accuracy: {np.mean(accuracies)}')
print(f'Standard deviation: {np.std(accuracies)}')
            

Training SVM for little vs index
58.0
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Training SVM for middle vs index
57.0
{'C': 100, 'gamma': 1e-06, 'kernel': 'rbf'}
Training SVM for middle vs little
58.0
{'C': 100, 'gamma': 1e-06, 'kernel': 'rbf'}
Training SVM for ring vs index
76.0
{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
Training SVM for ring vs little
81.0
{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
Training SVM for ring vs middle
78.0
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Training SVM for thumb vs index
63.0
{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Training SVM for thumb vs little
62.0
{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
Training SVM for thumb vs middle
62.0
{'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
Training SVM for thumb vs ring
85.0
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Average accuracy: 68.0
Standard deviation: 10.198039027185569


In [ ]:
# orig	  70.7


# svm	  reproduce 64
# 		  with_bad  66
# svm 	  gpt 		65

# mlp     reproduce 74
# mlp 	  gpt 		70



# tag = 'gpt4freq_all'
# Average accuracy: 68.2
# Standard deviation: 10.628264204469142



# 'gpt4freq_all'

# MLP 77.50